- the behavior planning team is responsible for providing guidance to the trajectory planner about what sorts of maneuvers they should plan trajectories for
- if you think about the overall flow of data in a self-driving car:
  - operating on the fastest time scales you have motion control
  - with a slightly lower frequency than that you have sensor fusion
  - just lower than that you have localization and trajectory planning
  - next is prediction
  - at the top of this diagram is behavior planning with the lowest update rate
- the inputs to behavior planning come from the prediction module and the localization module, both of which get their inputs from the sensor fusion
- the output from the behavior module goes directly to the trajectory planner, which also takes input from prediction and localization so that it can send trajectories to the motion controller


- everything inside the green box in the image below is the focus of this course and is commonly referred to as *path planning*
- in this lesson, you will learn what exactly has to happen in this comparatively long time span
  - intuitively this longtime span comes from the fact that the behavior planner has to incorporate a lot of data to make decisions about fairly long time horizons in the order of 10 seconds or even more

<img src="resources/sdc_data_flow.png"/>

# Understanding Output

- it's possible to suggest a wide variety of behaviors by specifying only a few quantities
- for example by specifying only a target lane, a target vehicle (to follow), a target speed, and a time to reach these targets, we can make suggestions as nuanced as "stay in your lane but get behind that vehicle in the right lane so that you can pass it when the gap gets big enough"


- look at the picture below and 5 potential json representations of output and see if you can match the json representation with the corresponding verbal suggestion

<img src="resources/understanding_output_quiz.jpg"/>

#### Output A
```json
{
    "target_lane_id" : 2,
    "target_leading_vehicle_id": 3,
    "target_speed" : null,
    "seconds_to_reach_target" : null,
}
```

#### Output B
```json
{
    "target_lane_id" : 3,
    "target_leading_vehicle_id": null,
    "target_speed" : 20.0,
    "seconds_to_reach_target" : 5.0,
}
```

#### Output C
```json
{
    "target_lane_id" : 2,
    "target_leading_vehicle_id": null,
    "target_speed" : 15.0,
    "seconds_to_reach_target" : 10.0,
}
```

#### Output D
```json
{
    "target_lane_id" : 2,
    "target_leading_vehicle_id": 2,
    "target_speed" : null,
    "seconds_to_reach_target" : 5.0,
}
```

#### Output E
```json
{
    "target_lane_id" : 1,
    "target_leading_vehicle_id": 2,
    "target_speed" : null,
    "seconds_to_reach_target" : 5.0,
}
```


- **Q:** Match the verbal description of each behavior to the corresponding json representation.
- **A:**
  - "Just stay in your lane and keep following the car in front of you" -- A
  - "Let's pass this car! Get in the left lane and follow that car" -- E
  - "Let's pass this car! But first we have to match speeds with the car in the left lane. Stay in this lane but get behind that car in the left lane and match their speed." -- D
  - "Whoa! This car we've been following is going too fast. Stop trying to follow it and just try to go the speed limit." -- C
  - "Get in the right lane soon." -- B

# The Behavior Problem

- imagine you're driving in a city with your friend and you have a goal that you're trying to get to
- you are sitting in the passenger seat and your friend is driving
- you plug that goal into Google Maps and you get some route that will get you to your destination
- the driver shouldn't be concerned about the details of what the route exactly is, because the driver trusts that you acting, as the navigator, will tell them what maneuvers they need to make and it's understood that they are just responsible for executing them
- in this situation, the responsibilities of the passenger are analogous to the responsibilities of a behavior planner in a self-driving car
- so if you tell your friend to go left, they will do that
- when you tell the driver to pass a slow vehicle, the driver won't worry about whether they have enough time to do so because they will assume that you have thought about that and that you're confident that they can make it
- but the navigator is not responsible for safety
- if you tell the driver to get back to the left lane but there is a vehicle in the way, you will assume that they will wait to move left until there is enough room to do so safely
- the navigator will also tell the driver when to keep their lane and when it is time to turn or change lanes or maybe adjust the speed
- finally, once you're arrived at your destination, you will tell the driver to stop


- to summarize, the behavior planner is currently a black box which takes as input a map of the world, a route to the destinations, and predictions about what other static and dynamic obstacles are likely to do, and it produces as output a suggested maneuver for the vehicle which the trajectory planner is responsible for reaching collision-free, smooth, and safe
- the goal of this lesson is to open up this black box and learn how to implement a behavior planner


- the responsibilities of the behavior module are to suggest maneuvers which are feasible, as safe as possible, legal, and efficient
- what the behavior planner is not responsible for are execution details and collision avoidance


- the approach to behavior planning we are going to teach in this lesson is the same one used by most of the vehicles, including the winning one in the DARPA Urban Challenge--in fact, it's the approach to behavior planning that Mercedes used in their Bertha Benz Drive, which you can see is capable of handling complex traffic situations like navigating intersections and dense urban traffic
- while this approach is not necessarily the most common anymore, for reasons we will discuss later, it is widely known in the field and it does give a very high-level understanding of what is going on in a behavior planner

# Finite State Machines

- in the previous demonstration, the navigator gave various suggestions about what to do next
- the instructions they gave were about things like changing lanes, following lanes, turning and so on but in fact there really aren't that many types of suggestions you would expect to hear from a navigator
- in this lesson we're going to teach an approach to behavior planning that uses something called a Finite State Machine to solve the behavior planning problem
- a Finite State Machine makes decisions based on a finite set of discrete states


- in this example below, when initialized, a Finite State Machine begins in some start state, let's call it $S0$
- any pair of states within the finite state machine can be connected by one or more transitions
- and sometimes there is a transition back to the same state
  - this is called a Self Transition
- not all transitions are necessarily possible
- for example, $S4$ doesn't transition to any other state
  - in the language of finite state machines this would be called an Accepting State
- for non-accepting states there can often be multiple potential successor states
- to decide which state to transition to next, a Finite State Machine needs to handle some sort of input and then use a state transition function to decide what state to go next

<img src="resources/finite_state_machines.png"/>

# Formalizing Finite State Machines

- let's consider a simple vending machine where everything costs 20 cents and let's say that this vending machine only takes nickels and dimes but nothing larger or smaller
- then we can model the state of this vending machine by the amount of money that's been deposited
  - the start state would be zero cents
  - from this state there are two things that can happen
    - we could put in a nickel, which would make the state five cents
    - we could put in a dime to take the state to 10 cents
  - the rest of the transitions are fairly straightforward until we think about what to do if we are in the 15 cent state and someone puts in a dime
    - we could just count that as 20 but let's say that this machine requires exact change so that a dime would just fall through and come out of the little tray at the bottom of the machine


- as you can see, finite state machines are pretty straightforward conceptually
- they have their strengths and weaknesses
- strengths:
  - finite state machines are very easy to reason about
    - they are basically self-documenting because they map the logical state of a system directly to the physical state
    - when a nickel goes into the vending machine the state changes to the state that is five cents bigger than the current one
  - they are maintainable
    - if we wanted to tweak this machine so that everything costs a quarter it would be pretty trivial to just add one more state
- weaknesses:
  - they are easily abused
    - if they aren't designed well to begin with or if the problem changes you can easily find yourself saying things like, "I hadn't considered that...", "Let's just add another state..." and this can lead to some sloppy code and sloppy logic which in practice means that finite state machines can be very difficult to maintain as the states base gets bigger

<img src="resources/finite_state_machines_vending_machine_example.png"/>

# FSM Intuition

<img src="resources/fsm_intuition_quiz_vending_machine.jpg"/>

- **Q:** Which state in the vending machine example was the start state?
- **A:** 0 cents.


- **Q:** Which state(s) were accepting states?
- **A:** 25 cents. An "accepting state" only has transitions into it and no transitions out of it.


- **Q:** If we wanted this machine to also accept pennies (1 cent coin) how many additional states would we have to add?
- **A:** 20. We would need one state for each integer between 0 and 25 (which is 26). We already have 6 states. So we would need 20 new states.

# States for Self Driving Cars

- when we're thinking about a simple vending machine it's easy to enumerate the states
- now let's consider the states we may want to model for car driving on a highway
- let's just throw out all the ideas we can come up with and then prune the list afterwards so we can show how we might think about coming up with a finite state machine from scratch
  - what happens if we are the only car on the road?
    - we would need a state for normal staying in your lane driving (keep lane)
    - if we are changing lanes, we want a state to represent that
    - or maybe two states since changing lanes to the left is different than to the right
  - what changes if there's a vehicle in front of us?
    - we might want to stay behind it, so we should have a state for that or we might want to pass it
    - I suppose, but isn't pass just a lane change to the left and then a lane change to the right?--do we really need in your state for that?
  - what changes if we had more cars?
    - I could imagine in this situation you might want to slow down so that you could merge into this gap here and then pass this car if it's going slow
    - we should really have a slow downstate which I guess means we will also need a speed upstate
    - we should probably add a stop state in case there is some emergency situation
    - I guess we could do all of that at once by having a keep target speed state
    - shouldn't speed just be dictated by the lane you're driving in and the speed limit?
    - what if we just add a prepare lane change state to represent when the ego vehicle is trying to go next to an empty gap in traffic before a lane change?
    - that could work but we should probably have a prepare lane change left and to prepare a lane change right, since those really are different maneuvers
    - we could also turn on the turn signals down


- the truth is there isn't a single correct set of states to choose from
- on one hand we want to keep our state space as small as possible for maintainability reasons, but on the other hand we want to make sure we have enough logical states to actually represent all the physical states that we care about
- it's not easy to find the perfect set of states for this scenario

# The States We'll Use

- in planning this lesson, we knew we wanted to focus on highway driving, but we still debated what states we should use for our finite state machine
- we quickly agreed on three (keep lane, change lane right, change lane left), since it seemed to us like these were the bare minimum and that a single lane change wouldn't suffice
- we also agreed to rule out a lot of states because most of these can be thought of as various implementations of the keep lane state
- but these last two, prepare lane change left and prepare lane change right, we were unsure about
  - eventually, we did decide to use these states for these lessons--so we should probably clarify what we mean by these states

<img src="resources/fsm_states_used_in_lecture.png"/>

- let's start by imagining what vehicle behavior would look like without these two prep states
- first, let me explain how this example will work
- up here we have a two-lane highway with traffic driving to the right
- this is our self-driving car (blue) and other traffic is shown in red
- as you watch this animation, you should imagine watching from a helicopter which is flying above the highway at the self-driving car's target speed
- these red trails indicate the relative speed of other vehicles
- in this situation, the vehicles in lane one are moving faster than we are while the vehicles in lane two are moving slower
- when we only have three states plus a fourth ready state, the finite state machine looks like this
- we'll assume that as we begin watching we are already in the keep lane state
- before we watch what happens to our vehicle when it uses this finite state machine, let me explain what these states mean


- the lane keep state attempts to stay in the current lane by staying near the center line for that lane
  - thinking in finite coordinates, we might just say that target $d$ for the vehicle is whatever the $d$ for the lane is
  - for the $s$ direction, keep lane state attempts to drive at the vehicle's target speed when that's feasible, but when it's not, it will try to drive at whatever speed is safest for the lane
- for lane changes the goal is to move from the initial lane to the target lane
  - the $d$ behavior is what you might expect to move left or right as appropriate so the target $d$ is the $d$ for whatever lane is to the left or right of the ego's current lane
  - for $s$, the same rules as lane keeping apply--the vehicle will try to drive at the target speed, but if that's not feasible, then it will drive at whatever speed is safe for the initial lane


- now, let's observe the vehicle's behavior as we move forward in time
- for the first few timesteps, we see that the ego-vehicle gets closer and closer to vehicle 2.2 while the vehicle in the left lane pass the ego-vehicle
- during this time the vehicle remains in the lane keep state since there isn't really enough gap to make a safe lane change
- note that at this point the ego-vehicle does begin to slow down even though it doesn't change state
- that's because the lane keep state contains this behavior
- when the target speed is no longer feasible, it will adjust speed to something safe for the lane
- at this point, the vehicle has decided that the lane change left is the best move
  - now, in the lane change state, you can see the vehicle has moved left in its lane but hasn't adjusted its speed
  - this is consistent with the rules we defined for lane change behavior
- at this point, the vehicle has crossed the dotted line, so its current lane is the left lane
- now it wants to transition back to the lane keep state to stay in this lane, and when it does that, it immediately starts accelerating so it can get to a speed that's safe for this lane
- right now it is moving at the same speed as other traffic in the left lane, which it can continue doing until it passes the vehicles on the right
- even though this worked, there were some problems
  - first, even when it was clear that there was a gap we could probably move into on the left, we had no way to tell the vehicle to try to get alongside that gap--we had to just wait for the gap to get close to us
  - second, this lane change wasn't all that safe--ideally, we would have been able to get closer to the left lane speed before actually making the lane change
  - third, it isn't clear when we would have turned on the turn signal in this scenario--the turn signal is the responsibility of the behavior team and ideally we would like to turn it on a few seconds before we actually start changing lanes


- using this finite state machine, it would have been hard to do that
- to address that problem, we can introduce a prepare for lane change state
- in this state we do whatever we can to prepare for a lane change left or right, which means in the $d$ directions we still can stay in this current lane but in the $s$ direction we tried to match the position and speed of some gap in one of the adjacent lanes
- this is also when we would turn on the appropriate signal
- we also changed the finite state machine to look like this
- and note that the only way to change lanes is to first prepare for a lane change
- let's see what that would look like
- at the first timestep we can decide to prepare for a lane change by tracking this gap over here, which allows us to immediately start slowing down
- at this point, the car may begin slowly increasing speed to get closer to the left lane speed, and at this point the vehicle has reached the same speed as the left lane traffic and is in a good position to execute a lane change, which it begins here
- from here on, everything looks pretty similar to before
- with these added states, we're now able to perform a lane change more safely and efficiently

# Inputs to Transition Functions

- you just saw how the states we choose to use can impact the behavior of the vehicle
- but deciding how those states transition and what inputs the transition functions use is crucial to the actual implementation of a finite-state machine
- for the example with the vending machine, the only input was the coin
- the self-driving car is more complicated


- **Q:** What data will we need to pass into our transition function as input? Check all that apply.
- **A:** Predictions, Map, Speed Limit, Localization Data, Current State. We need everything except for the previous state.

# Behavior Planning Pseudocode

- one way to implement a transition function is by generating rough trajectories for each accessible "next state" and then finding the best
- to "find the best" we generally use cost functions
- we can then figure out how costly each rough trajectory is and then select the state with the lowest cost trajectory


- we'll discuss this in more detail later, but first read carefully through the pseudocode below to get a better sense for how a transition function might work

```python
def transition_function(predictions, current_fsm_state, current_pose, cost_functions, weights):
    # only consider states which can be reached from current FSM state.
    possible_successor_states = successor_states(current_fsm_state)

    # keep track of the total cost of each state.
    costs = []
    for state in possible_successor_states:
        # generate a rough idea of what trajectory we would
        # follow IF we chose this state.
        trajectory_for_state = generate_trajectory(state, current_pose, predictions)

        # calculate the "cost" associated with that trajectory.
        cost_for_state = 0
        for i in range(len(cost_functions)) :
            # apply each cost function to the generated trajectory
            cost_function = cost_functions[i]
            cost_for_cost_function = cost_function(trajectory_for_state, predictions)

            # multiply the cost by the associated weight
            weight = weights[i]
            cost_for_state += weight * cost_for_cost_function
            costs.append({'state' : state, 'cost' : cost_for_state})

    # Find the minimum cost state.
    best_next_state = None
    min_cost = 9999999
    for i in range(len(possible_successor_states)):
        state = possible_successor_states[i]
        cost  = costs[i]
        if cost < min_cost:
            min_cost = cost
            best_next_state = state 

    return best_next_state
```

# Create a Cost Function - Speed Penalty

- a key part of getting transitions to happen when we want them to is the design of reasonable cost functions
- we want to penalize wrong and reward the right things
- I'm going to work through an example of one way you might think about designing a cost function


- let's consider how we would design a cost function for vehicle speed
- on one hand, we want to get to our destination quickly, but on the other hand, we don't want to break the law
- an essential quantity we have to control is the desired velocity of the car
- some velocities are more beneficial, some are even illegal
- let's fill in this graph and try to assign some costs to every velocity

<img src="resources/speed_cost.png"/>

- for the sake of simplicity, let's assume that all of the cost functions will have an output between zero and one
- we will adjust the importance of each cost function later by adjusting the weights
- let's say the speed limit for the road we are on is here
- we know that if we're going well above the speed limit, that should be maximum cost (circle on dotted line)
- and maybe we want to set an ideal zero cost speed that's slightly below the speed limit so that we have some buffer (circle on x-axis)
- then we can think about how much we want to penalize not moving at all
- obvously, not moving is bad, but maybe not as bad as breaking the speed limit, so we would put it here (y-axis)
- to keep it simple, we could just say there is a linear cost between zero and the target speed
- since breaking the law is a binary thing, let's just say any speed greater than or equal the speed limit has maximal cost
- again, we can arbitrarily connect these points with a linear function and the flat maximum cost for anything above the speed limit


- in practice, we might actually want to parametrize some of these quantities so that we could later adjust them until we got the right behavior
- first, we might define a parameter called stop cost for the zero-velocity case and a parameter called buffer velocity which would probably be a few miles per hour
- then, our overall cost function has three domains
  - if we're going less than the target speed
  - if we are above the speed limit (the cost is just one)
  - if we are between

# Example Cost Function - Lane Change Penalty

<img src="resources/lane_change_cost_function_example.png"/>

- in the image above, the blue self driving car (bottom left) is trying to get to the goal (gold star)
- it's currently in the correct lane but the green car is going very slowly, so it considers whether it should perform a lane change (LC) or just keep lane (KL)
- these options are shown as lighter blue vehicles with a dashed outline


- if we want to design a cost function that deals with lane choice, it will be helpful to establish what the relevant variables are
- in this case, we can define:
  - $\Delta s = s_G - s$ how much distance the vehicle will have before it has to get into the goal lane
  - $\Delta d = d_G - d_{LC/KL}$ the lateral distance between the goal lane and the options being considered
    - in this case $\Delta d_{KL} = d_G - d_{KL}$ would be zero and $\Delta d_{LC} = d_G - d_{LC}$ would not


- **Q:** First, thinking only about $\Delta d$: Would we prefer the absolute value of $\Delta d$ to be big or small?
- **A:** Small. In general we'd prefer to be closer to the goal lane. Which means the cost should be bigger when $\Delta d$ is bigger. Mathematically we could say "Cost associated with lane choice should be proportional to $\Delta d$.


- **Q:** Now let's think about how $s$ factors into our considerations of lane cost. Should costs associated with lane change be more important when we are far from the goal (in $s$ coordinate) or close to the goal?
- **A:** Close. When we're far from the goal we have lots of time to get into the goal lane. As we get closer things get more urgent. Mathematically we could say "costs associated with lane choice should be inversely proportional to $\Delta s$".


- so we want a cost function that penalizes large $|\Delta d|$ and we want that penalty to be bigger when $\Delta s$ is small
- furthermore, we want to make sure that the maximum cost of this cost function never exceeds $1$ and that the minimum never goes below $0$


- **Q:** Which of the following proposals meet the criteria we want (assume $\Delta s$ is always positive)?
  - Option 1: $\text{cost} = |\Delta d| + \frac{1}{\Delta s}$
  - Option 2: $\text{cost} = \frac{|\Delta d|}{\Delta s}$
  - Option 3: $\text{cost} = 1 - e^{- \frac{|\Delta d|}{\Delta s}}$
- **A:** Option 3. When the ratio of $d$ to $s$ is big, this quantity approaches maximum cost (1) but when it's small, it approaches minimum cost (0).


- in this example, we found that the ratio $\dfrac{|\Delta d|}{\Delta s}$ was important
- if we call that ratio $x$ we can then use that ratio in any function with bounded range
- these functions tend to be useful when designing cost functions
- these types of functions are called Sigmoid Functions
  - you can learn more in the [Wikipedia article](https://en.wikipedia.org/wiki/Sigmoid_function) if you're interested

# Implement a Cost Function in C++

- in the previous quizzes, you designed a cost function to choose a lane when trying to reach a goal in highway driving: $\text{cost} = 1 - e^{- \dfrac{|\Delta d|}{\Delta s}}$
- here, $\Delta d$ was the lateral distance between the goal lane and the final chosen lane, and $\Delta s$ was the longitudinal distance from the vehicle to the goal


- in this quiz, we'd like you to implement the cost function in C++, but with one important change
- the finite state machine we use for vehicle behavior also includes states for planning a lane change right or left (PLCR or PLCL), and the cost function should incorporate this information
- we will provide the following four inputs to the function:
  - intended lane: the intended lane for the given behavior
    - for PLCR, PLCL, LCR, and LCL, this would be the one lane over from the current lane
  - final lane: the immediate resulting lane of the given behavior
    - for LCR and LCL, this would be one lane over
  - the $\Delta s$ distance to the goal
  - the goal lane


- your task in the implementation will be to modify $|\Delta d|$ in the equation above so that it satisifes:
  - $|\Delta d|$ is smaller as both intended lane and final lane are closer to the goal lane
  - the cost function provides different costs for each possible behavior: KL, PLCR/PLCL, LCR/LCL
  - the values produced by the cost function are in the range $0$ to $1$

- code is available in `01_implement_a_cost_function/`

# Implement a Second Cost Function in C++

- in most situations, a single cost function will not be sufficient to produce complex vehicle behavior
- in this quiz, we'd like you to implement one more cost function in C++
- we will use these two C++ cost functions later in the lesson
- the goal with this quiz is to create a cost function that would make the vehicle drive in the fastest possible lane, given several behavior options
- we will provide the following four inputs to the function:
  - target speed: currently set as $10$ (unitless), the speed at which you would like the vehicle to travel
  - intended lane: the intended lane for the given behavior
    - for PLCR, PLCL, LCR, and LCL, this would be the one lane over from the current lane
  - final lane: the immediate resulting lane of the given behavior
    - for LCR and LCL, this would be one lane over
  - vector of lane speeds, based on traffic in that lane: $\{6, 7, 8, 9\}$


- your task in the implementation will be to create a cost function that satisifes:
  - the cost decreases as both intended lane and final lane are higher speed lanes
  - the cost function provides different costs for each possible behavior: KL, PLCR/PLCL, LCR/LCL
  - the values produced by the cost function are in the range $0$ to $1$

- code is available in `02_implement_a_second_cost_function/`

# Cost Function Design and Weight Tweaking

- designing cost functions is difficult and getting them all to cooperate to produce reasonable vehicle behavior is hard
- some of the difficulties associated with cost functions design include:
  - solving new problems without "unsolving" old ones
    - when you're working on a self-driving car, you may find that the vehicle is behaving reasonably well except for some particular situations
    - maybe it's not being aggressive enough about making left turns at traffic lights so, in an effort to solve this problem, you either add new cost functions, tweak existing ones, or modify the weights but every time you do, there's a chance that you will introduce some breaking change into something that already works
    - in practice, we solve this through regression testing, where we define some set of situations, each of which has an expected behavior
    - then, whenever we make a change, we simulate the vehicle in all of our test cases and make sure that it still behaves as expected
  - balancing costs of drastically different magnitudes
    - we want to get to our destination efficiently, but if we are in a situation where safety is an issue, we want to solve that problem and not think about efficiency at all
    - one way to do that is to have weights which reflect the type of problem the cost function addresses
    - we want to most heavily penalize any behavior which simply isn't possible due to physics, then we want to think about safety, legality, comfort
      - only once those are satisfied do we want to think about efficiency
    - we also may want to adjust the relative importance of these weights depending on situation
      - if a light turns red, for example, legality becomes a much more relevant concern than when we engage in normal highway driving
  - reasoning about individual cost functions
    - ideally, each cost function will serve a very specific responsibility, which is something we didn't do in our earlier example of a speed cost function
    - we were trying to balance our desire to drive quickly, which has to do with efficiency, with our desire to not exceed the speed limit, which is legality
    - in practice, we might want to define several cost functions associated with vehicle speed in which case we might have a binary cost function which just checks to see if we are breaking the speed limit and the continuous cost function which pulls us towards our target speed
    - by assigning each cost function to a very specific role, like safety versus legality versus efficiency, we can then standardize the output of all cost functions to be between $-1$ and $1$
    - additionally, it's helpful to parametrize whenever possible
      - this allows us to use some parameter optimization technique like gradient descent along with our set of regression tests to programmatically tweak our cost functions
    - finally, thinking in terms of vehicle state is helpful
      -  things we can indirectly control about our vehicle are its position, velocity, and acceleration
      - it can be helpful to keep these in mind when coming up with cost functions

<img src="resources/cost_function_design_class_feasibility.png"/>

- say we want to think about the following classes of cost functions
- to make it easier, to keep everything straight, let's think in terms of position, velocity, and acceleration
- if this is all starting to feel like it's getting pretty complex, well, you're right
- it's pretty hard to avoid this exploding complexity when using finite state machines--partially that's because of the finite state machine itself, but we're also trying to solve a very hard problem and some complexity is unavoidable no matter what solution approach you take

# Cost Function Matching

- consider the following cost functions, what purpose does each serve?
  - Cost Function 1
    - $\begin{cases} 1 & \ddot{s} \geq a\_{\text{max}} \\ 0 & \ddot{s} < a\_{\text{max}} \end{cases}$
  - Cost Function 2
    - $\begin{cases} 1 & d \geq d\_{\text{max}} \\ 1 & d \leq d\_{\text{min}} \\ 0 & d\_{\text{min}} < d < d\_{\text{max}} \end{cases}$
  - Cost Function 3
    - $\begin{cases} 1 & \dot{s} \geq v\_{\text{speed limit}} \\ 0 & \dot{s} < v\_{\text{speed limit}} \end{cases}$
  - Cost Function 4
    - $\dfrac{1}{1 + e^{-(d - d\_{\text{lane center}})^2}}$
  - Cost Function 5
    - $(\text{lane number} - \text{target lane number}) ^ 2$


- **Q:** Match the verbal descriptions of each cost function to the appropriate number from above.
- **A:**
  - Penalizes trajectories that do not stay near the center of the lane. -- 4
  - Penalizes trajectories that attempt to accelerate at a rate which is not possible for the vehicle. -- 1
  - Rewards trajectories that stay near the target lane. -- 5
  - Penalizes trajectories that drive off the road. -- 2
  - Penalizes trajectories that exceed the speed limit. -- 3

# Scheduling Compute Time

- by now you might be able to guess why the Behavior Module updates on a lower frequency than for example the Trajectory Module
- this is due to the fact that the high level decisions made in the behavior module spend a longer time horizon and just don't change as frequently
- but the Trajectory Module does still count on our decisions and it's important that the overall system architecture doesn't allow for a comparatively slow module like the behavior planner to clog up the proper functioning of the other faster components

<img src="resources/scheduling_compute_time.png"/>

- let's take a second to talk about what's known as a scheduling problem and how it can be handled in the self-driving car
- this diagram shows what happens during two processing cycles of the Behavior Module
- as you can see, the Prediction Module updates with a higher frequency than Behavior
- Trajectory is even higher and so on...
- but focus your attention on what happens after behavior has completed its first cycle
- to begin its second cycle, the behavior module needs data from prediction and localization
- for localization, it's easy in theory since at this instant it will have some fresh data and behavior could just use that
- but what about for prediction--it's actually right in the middle of an update cycle at this instant
- should behavior just wait until prediction is done?--no, if we start waiting then we block up the pipeline for downstream components
- the answer is to use data from here and accept that it's a little stale

# Implement Behavior Planner in C++

- in this exercise you will implement a behavior planner and cost functions for highway driving
- the planner will use prediction data to set the state of the ego vehicle to one of 5 values and generate a corresponding vehicle trajectory:
  - "KL" - Keep Lane
  - "LCL" / "LCR"- Lane Change Left / Lane Change Right
  - "PLCL" / "PLCR" - Prepare Lane Change Left / Prepare Lane Change Right


- the objective of the quiz is to navigate through traffic to the goal in as little time as possible
- note that the goal lane and $s$ value, as well as the traffic speeds for each lane, are set in `main.cpp`
- since the goal is in the slowest lane, in order to get the lowest time, you'll want to choose cost functions and weights to drive in faster lanes when appropriate
- we've provided two suggested cost functions in `cost.cpp`


- instructions
  - implement the `choose_next_state` method in the `vehicle.cpp` class
    - you can use the Behavior Planning Pseudocode concept as a guideline for your implementation
    - in this quiz, there are a couple of small differences from that pseudocode: you'll be returning a best trajectory corresponding to the best state instead of the state itself
    - additionally, the function inputs will be slightly different in this quiz than in the classroom concept
    - for this part of the quiz, we have provided several useful functions:
      - `successor_states()` - uses the current state to return a vector of possible successor states for the finite state machine
      - `generate_trajectory()` - returns a vector of Vehicle objects representing a vehicle trajectory, given a state and predictions
        - note that trajectory vectors might have size $0$ if no possible trajectory exists for the state
      - `calculate_cost()` - included from `cost.cpp`, computes the cost for a trajectory
  - choose appropriate weights for the cost functions in `cost.cpp` to induce the desired vehicle behavior
    - two suggested cost functions have been implemented based on previous quizzes, but you are free to experiment with your own cost functions
    - the `get_helper_data()` function in `cost.cpp` provides some preprocessing of vehicle data that may be useful in your cost functions
    - see if you can get the vehicle to move into a fast lane for a portion of the track and then move back to reach the goal!

- code is available in `03_implement_behavior_planner_in_cpp/`

# Conclusion

- in this lesson, you learned about finite state machines which are a solution to the behavior planning problem
- finite state machines generally do well for small state spaces, like when you're on a highway, for example
- for more complex scenarios like urban driving, other approaches might be more suitable